In [ ]:
import os, sys
import cProfile
sys.path.append(os.path.join(os.path.abspath(''), '../'))

from databases.ExperimentDb import (
    db, 
    init_from_default_db, 
    ImageOnlyShader, 
    ImageOnlyExperiment,
    AugmentationType,
    packBytesToSpv,
    packSpvToBytes,
    ErrorType
    )
from databases.AugmentationDb import dbA, init_from_default_dbA, Augmentation
import cliFuncs.augmentImageOnly as augmentImageOnly
import cliFuncs.runImageOnlyTrace as runImageOnlyTrace
import cliFuncs.runImageOnly as runImageOnly
import dataclasses
from utils.AugmentationManager import (
    AugManager,
    AugManagerConfig,
    augmentAll,
    augmentAllNoPall
)
from tqdm import tqdm
from joblib import Parallel, delayed
import vkExecute
import datetime
import argparse

import matplotlib.pyplot as plt
import numpy as np

init_from_default_db()
init_from_default_dbA()

### Augment random shaders

In [ ]:
# init database and args

@dataclasses.dataclass
class Argument:
    shader_id = ""              # shader id for query
    environment_id = 1         # environment id for query
    env_comment = ""            # environment comment for query
    save_images = False         # if true, save images to disk
    no_subprocess = False       # if true, do not use subprocess
    disable_timeout = False     # if true, do not use timeout
    timeout = 120               # timeout for each render
    augment_flag = "O"           # augment flag, if random_times is not 0, this flag will be ignored
    random_times = 0            # random augment times
    aug_parent_flag = ""        # parent augment flag for query
    limit = 20                 # limit for query
    
args = Argument()

In [ ]:
augmentImageOnly.cliAugment(db, args)

### Create databass

In [ ]:
# dbA.create_tables([Augmentation])

In [ ]:
# query = ImageOnlyShader.select()

# for shader in query:
#     augmentation = Augmentation.create(
#         augmentation = AugmentationType.NONE,
#         augmentation_annotation = None,
#         fragment_spv = shader.fragment_spv,
#         shader_id = shader.shader_id,
#         depth = 0,
#         parent = None,
#         dis_from_last = 0,
#         dis_ratio_from_last = 0,
#         dis_from_origin = 0,
#         dis_ratio_from_origin = 0
#     )


In [ ]:
parser = argparse.ArgumentParser()
runImageOnly.register(parser)
args = parser.parse_args("")

args.only_compile = True

with db.atomic():
  runImageOnly.cliRun(db, args)

### Augment selected pass

In [ ]:
query = Augmentation.select().where(
    Augmentation.depth == 0,
    Augmentation.id != 4917,
    Augmentation.id != 5399
    )


augmentAll([AugmentationType.INST_DESC_IDX_CHECK], None, query)

### Export all

In [ ]:
env = 1
width = 1024
targetAug = AugmentationType.O
targetDepth = 1

query = Augmentation.select().where(Augmentation.depth == 0)
augNodesSrc = []
augNodesDes = []

for aug in tqdm(query):
    dest = ImageOnlyExperiment.select().where(
        ImageOnlyExperiment.augmentation == AugmentationType.NONE,
        ImageOnlyExperiment.environment == env,
        ImageOnlyExperiment.width == width,
        ImageOnlyExperiment.shader_shadertoy_id == aug.shader_id,
    )
    if dest.count() == 0 or dest.count() > 1:
        print("error")
        continue
    else:
        augNodesSrc.clear()
        augNodesDes.clear()
        src = aug
        des = dest[0]
        while src.children.count() > 0:
            for child in src.children:
                if child.depth > targetDepth:
                    continue
                if targetAug != None and child.depth == targetDepth and child.augmentation != targetAug:
                    continue
                
                augNodesSrc.append(child)

                repeated = False
                for desChild in des.children:
                    if desChild.augmentation == child.augmentation \
                        and desChild.augmentation_annotation == child.augmentation_annotation:
                        augNodesDes.append(desChild)
                        repeated = True
                        break
                if repeated:
                    continue

                shader = ImageOnlyShader.create(
                    shader_id = child.shader_id,
                    fragment_spv = child.fragment_spv,
                    )
                shader.save()

                expr = ImageOnlyExperiment.create(
                    shader_shadertoy_id = src.shader_id,
                    shader = shader,
                    resource = des.resource,
                    environment = env,
                    width = des.width,
                    height = des.height,
                    augmentation = child.augmentation,
                    augmentation_annotation = child.augmentation_annotation,
                    parent = des,
                    time = datetime.datetime.now(),
                    num_cycles = des.num_cycles,
                    num_trials = des.num_trials,
                    errors = ErrorType.AUGMENTED_BUT_NOT_RUN,
                    measurement = des.measurement,
                    results = ""
                    )
                expr.save()
                augNodesDes.append(expr)
            
            src = augNodesSrc.pop(0)
            des = augNodesDes.pop(0)

### Trace All
* This will run all shaders which were augmented but not run.

In [ ]:
parser = argparse.ArgumentParser()
runImageOnlyTrace.register(parser)
args = parser.parse_args("")

args.augment_trace = True

runImageOnlyTrace.cliTrace(db, args)

### Run All
* This will run all selected shaders which were augmented but not run.
* After successfully rendering the error will be set to None

In [ ]:
parser = argparse.ArgumentParser()
augmentImageOnly.register(parser)
args = parser.parse_args("")

args.augment_flag = "None"

with db.atomic():
  augmentImageOnly.cliAugmentRun(db, args)

### Tools for checking fragmentSpv

In [ ]:
# Export fragment spv to txt file
def exportFragmentSpvToTxt(fragmentSpv, filename):
    with open(filename, 'w') as file:
        file.write(fragmentSpv)

def diffAugShaderFromAncestor(aug: Augmentation):
    thisAug = aug
    while aug.parent is not None:
        aug = aug.parent
    ancestorAug = aug
    spvProc = vkExecute.SpvProcessor()
    spvProc.loadSpv(packBytesToSpv(ancestorAug.fragment_spv))
    ancestorShader, err = spvProc.disassemble()
    exportFragmentSpvToTxt(ancestorShader, "ancestorShader.txt")
    spvProc.loadSpv(packBytesToSpv(thisAug.fragment_spv))
    thisShader, err = spvProc.disassemble()
    exportFragmentSpvToTxt(thisShader, "thisShader.txt")

def diffAugShaderFromParent(aug: Augmentation):
    if aug.parent is None:
        return
    spvProc = vkExecute.SpvProcessor()
    spvProc.loadSpv(packBytesToSpv(aug.parent.fragment_spv))
    parentShader, err = spvProc.disassemble()
    exportFragmentSpvToTxt(parentShader, "parentShader.txt")
    spvProc.loadSpv(packBytesToSpv(aug.fragment_spv))
    thisShader, err = spvProc.disassemble()
    exportFragmentSpvToTxt(thisShader, "thisShader.txt")

diffAugShaderFromParent(Augmentation.get_by_id(102510))

In [ ]:
query0 = Augmentation.select().where(Augmentation.augmentation == AugmentationType.OS)
query1 = Augmentation.select().where(Augmentation.augmentation == AugmentationType.O)
fragSpvLen0 = []
fragSpvLen1 = []
for aug in query0:
    fragSpvLen0.append(len(aug.fragment_spv))
for aug in query1:
    fragSpvLen1.append(len(aug.fragment_spv))

avg0 = np.mean(fragSpvLen0)
avg1 = np.mean(fragSpvLen1)
std0 = np.std(fragSpvLen0)
std1 = np.std(fragSpvLen1)

print("avg0: ", avg0)
print("avg1: ", avg1)
print("std0: ", std0)
print("std1: ", std1)